In [2]:
!pip install implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 58.2 MB/s eta 0:00:00


In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [4]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [5]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [10]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[834631, 856942, 866227, 869465, 920200, 93113..."
1,2,"[857966, 869543, 1031021, 1129249, 1133393, 80..."


Задание 1. Weighted Random Recommendation
Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж

Можно сэмплировать товары случайно, но пропорционально какому-либо весу
Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [ ]:
def weighted_random_recommendation(items_weights, n=5):

    items_weights = np.array(items_weights)

    p_list = np.linspace(0.01, 0.9, n).tolist()

    recs = np.random.choice(items_weights, size=n, replace=False)

    recs = np.random.choice(recs, size=n, p=[0.3, 0.25, 0.2, 0.15, 0.1], replace=False)

    return recs.tolist()

In [25]:
%%time

items = data_train.item_id.unique()

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items, n=5))
result.head(2)

CPU times: user 2.01 s, sys: 0 ns, total: 2.01 s
Wall time: 2.01 s


,user_id,actual,weighted_random_recommendation
0,1,"[834631, 856942, 866227, 869465, 920200, 93113...","[924352, 9526274, 9837014, 819264, 6773205]"
1,2,"[857966, 869543, 1031021, 1129249, 1133393, 80...","[1084607, 1473658, 1101822, 1012906, 9884204]"


Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [27]:
result = pd.read_csv('predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [28]:
def precision_at_k(recommended_list, bought_list, k=5):

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]

    flags = np.isin(bought_list, recommended_list)

    precision = flags.sum() / len(recommended_list)


    return precision

In [31]:
bought_list = result['actual']

In [32]:
recommended_list = result['random_recommendation']
precision_at_k(recommended_list, bought_list, k=5)

0.0

In [33]:
recommended_list = result['popular_recommendation']
precision_at_k(recommended_list, bought_list, k=5)

0.0

In [34]:
recommended_list = result['itemitem']
precision_at_k(recommended_list, bought_list, k=5)

0.0

In [35]:
recommended_list = result['cosine']
precision_at_k(recommended_list, bought_list, k=5)

0.0

In [37]:
recommended_list = result['tfidf']
precision_at_k(recommended_list, bought_list, k=5)

0.0

In [38]:
recommended_list = result['own_purchases']
precision_at_k(recommended_list, bought_list, k=5)

0.0